In [1]:
from tqdm.auto import tqdm
import sqlite3

In [8]:
conn = sqlite3.connect('cuis.db')
cursor = conn.cursor()

In [9]:
create_cuis_query = """
CREATE TABLE IF NOT EXISTS cuis (
    cui TEXT PRIMARY_KEY,
    ignored BOOLEAN,
    canonical_name TEXT
)
"""
create_rels_query = """
CREATE TABLE IF NOT EXISTS relations (

pmid_source varchar(15),
sentence_source TEXT,
first_sign_cui TEXT,
first_sign_modifier INT CHECK (first_sign_modifier IN (-1,0,1)),
second_sign_cui TEXT,
second_sign_modifier INT CHECK (second_sign_modifier IN (-1,0,1)),

relation_type TEXT CHECK (relation_type IN ("cause","correlation")),
FOREIGN KEY (first_sign_cui) REFERENCES cuis (cui),
FOREIGN KEY (second_sign_cui) REFERENCES cuis (cui)

PRIMARY KEY (sentence_source, first_sign_cui, second_sign_cui)
)
"""


In [10]:
cursor.execute("DROP TABLE IF EXISTS relations")
cursor.execute("DROP TABLE IF EXISTS cuis")
cursor.execute(create_cuis_query)
cursor.execute(create_rels_query)

In [16]:
def read_mrconso(path=None):
    res = set()
    if path is None:
        path = "/Users/ldorigo/Documents/UMLS/Documents/2018AB/META/MRCONSO.RRF"
    with open(path,"r") as mrc:
        for line in tqdm(mrc, total = 10304539):
            els = line.split("|")[:-1]
            ## only keep english terms:
            if els[1] != 'ENG':
                continue
            ## ignore obsolete terms
            if els[16] != 'N':
                continue
            ## only keep the preferred terms
            if els[6] != 'Y':
                continue
                
            res.add((els[0],None,els[14]))
    return list(res)
            

In [17]:
vals = read_mrconso()

In [19]:
cursor.executemany('INSERT INTO cuis VALUES (?,?,?)',vals)

In [20]:
conn.commit()

In [21]:
del vals